In [1]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.metrics.distance import edit_distance
nltk.download('stopwords')
import string
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#import dataset
df = pd.read_csv('dataset_tundapemilu.csv')
df.tail(10)

,date,username,tweet
105,2022-04-02 09:35:18,ZAEffendy,"Aksi Demo Jumat, 1 April 2022 di Jakarta #Maha..."
106,2022-04-02 09:31:25,elHarist,"Lebaran ga kebeli baju baru gpp, asal pemilu d..."
107,2022-04-02 09:30:20,ZAEffendy,"Jumat, 1 April 2022 di Jakarta Aksi Demo #Mah..."
108,2022-04-02 09:27:35,ZAEffendy,"Jumat, 1 April 2022 Aksi Demo di Jakarta #Maha..."
109,2022-04-02 09:23:46,ZAEffendy,"Kemarin Jumat, 1 April 2022 kembali #Mahasiswa..."
110,2022-04-02 08:20:45,Benalumati,@m1n4_95 Pantesan getol banget mempertahankan ...
111,2022-04-02 05:51:43,Pakamkalam,@MprAldo @AbdlJbrl Mau shaum Sabtu atau Ahad t...
112,2022-04-02 05:50:02,moleriz,#tolakPenundaanPemilu #Tolak3Periode
113,2022-04-02 04:51:57,IwanSentani,"Ngurus hal receh ora gableg, ngotot mau berkua..."
114,2022-04-02 04:22:35,AdhizBuana,😵🥴😲 #Tolak3Periode #tolakPenundaanPemilu #To...


In [34]:
#hanya mengambil data untuk dilabelin
df3=df['tweet']
df3

0      Apa dia ngerti ? #PemiluSerentak2024 #TolakTeg...
1      #TolakTegasPenundaanPemilu  #tolakPenundaanPemilu
2      Suara Sumbang ancam Konstitusi, Marwah Desa di...
3      Banyak tong. Bahkan lbh baik. Siapa? Buka mata...
4      Jokowi klaim, usulan 3 periode dari masyarakat...
                             ...                        
110    @m1n4_95 Pantesan getol banget mempertahankan ...
111    @MprAldo @AbdlJbrl Mau shaum Sabtu atau Ahad t...
112                #tolakPenundaanPemilu  #Tolak3Periode
113    Ngurus hal receh ora gableg, ngotot mau berkua...
114    😵🥴😲 #Tolak3Periode  #tolakPenundaanPemilu  #To...
Name: tweet, Length: 115, dtype: object

In [37]:
#Conver csv
df3.to_csv("jadilabelin.csv", index=False)  

In [5]:
#import dataset sudah dikasih label
df= pd.read_excel('sudahkasihlabel.xlsx')
df

,Label,tweet
0,Netral,Apa dia ngerti ? #PemiluSerentak2024 #TolakTeg...
1,Netral,#TolakTegasPenundaanPemilu #tolakPenundaanPemilu
2,Negatif,"Suara Sumbang ancam Konstitusi, Marwah Desa di..."
3,Negatif,Banyak tong. Bahkan lbh baik. Siapa? Buka mata...
4,Negatif,"Jokowi klaim, usulan 3 periode dari masyarakat..."
...,...,...
110,Negatif,@m1n4_95 Pantesan getol banget mempertahankan ...
111,Netral,@MprAldo @AbdlJbrl Mau shaum Sabtu atau Ahad t...
112,Netral,#tolakPenundaanPemilu #Tolak3Periode
113,Negatif,"Ngurus hal receh ora gableg, ngotot mau berkua..."


In [4]:
def cleansing_text(text):
    # replace RT tag
    text = re.sub('RT\s', '',text)
    # replace @_username
    text = re.sub('\B@\w+', '', text)
    # replace emoji dengan text
    text = emoji.demojize(text)
    # replace URL
    text = re.sub('(http|https):\/\/\S+', '', text)
    # replace #something
    text = re.sub('#+', '', text)
    # konversi huruf kapital ke huruf kecil semua
    text = text.lower()
    # replace kata yang berulang-ulang ('oooooo' menjadi '00')
    text = re.sub(r'(.)\1+', r'\1\1', text)
    # replace punctuation repetition dengan single occurance ('!!!!!!' ,enjadi '!')
    text = re.sub(r'[\?\.\!]+(?=[\?.\!])', '', text)
    # menhilangkan angka dan spesial karakter, hanya mengambil alfabet saja
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # replace contractions 
    text = contractions.fix(text)
    return text

df['tweet'] = df['tweet'].apply(lambda x: cleaning_text(x))
df.head(10)

NameError: name 'cleaning_text' is not defined

In [51]:
#Tokenization/Memisahkan dretan kata dalam kalimat
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tokenization'] = df['tweet'].apply(lambda x: tokenization(x.lower()))
df.head(10)

,Label,tweet,Tokenization
0,Netral,Apa dia ngerti,"[apa, dia, ngerti]"
1,Netral,,[]
2,Negatif,Suara Sumbang ancam Konstitusi Marwah Desa dii...,"[suara, sumbang, ancam, konstitusi, marwah, de..."
3,Negatif,Banyak tong Bahkan lbh baik Siapa Buka matamu ...,"[banyak, tong, bahkan, lbh, baik, siapa, buka,..."
4,Negatif,Jokowi klaim usulan periode dari masyarakat Ma...,"[jokowi, klaim, usulan, periode, dari, masyara..."
5,Negatif,Istana ndableg kita tetap demo,"[istana, ndableg, kita, tetap, demo]"
6,Positif,Reformasi jilid menanti Gantian Periode,"[reformasi, jilid, menanti, gantian, periode]"
7,Positif,Maju terus mhsiswa seluruh bersama rakyat Panj...,"[maju, terus, mhsiswa, seluruh, bersama, rakya..."
8,Negatif,Ormas dan dukungan abal masih akan ada lagi LB...,"[ormas, dan, dukungan, abal, masih, akan, ada,..."
9,Negatif,Semakin dahsyat untuk mencapai nafsu kekuasaan,"[semakin, dahsyat, untuk, mencapai, nafsu, kek..."


In [52]:
#Stop Removal/Pemilihan kata
stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

df['Stop_Removal'] = df['Tokenization'].apply(lambda x: remove_stopwords(x))
df.head(10)

,Label,tweet,Tokenization,Stop_Removal
0,Netral,Apa dia ngerti,"[apa, dia, ngerti]",[ngerti]
1,Netral,,[],[]
2,Negatif,Suara Sumbang ancam Konstitusi Marwah Desa dii...,"[suara, sumbang, ancam, konstitusi, marwah, de...","[suara, sumbang, ancam, konstitusi, marwah, de..."
3,Negatif,Banyak tong Bahkan lbh baik Siapa Buka matamu ...,"[banyak, tong, bahkan, lbh, baik, siapa, buka,...","[tong, lbh, buka, matamu, gimmick, akalan, kau..."
4,Negatif,Jokowi klaim usulan periode dari masyarakat Ma...,"[jokowi, klaim, usulan, periode, dari, masyara...","[jokowi, klaim, usulan, periode, masyarakat, m..."
5,Negatif,Istana ndableg kita tetap demo,"[istana, ndableg, kita, tetap, demo]","[istana, ndableg, demo]"
6,Positif,Reformasi jilid menanti Gantian Periode,"[reformasi, jilid, menanti, gantian, periode]","[reformasi, jilid, gantian, periode]"
7,Positif,Maju terus mhsiswa seluruh bersama rakyat Panj...,"[maju, terus, mhsiswa, seluruh, bersama, rakya...","[maju, mhsiswa, rakyat, umur, perjuangan, peri..."
8,Negatif,Ormas dan dukungan abal masih akan ada lagi LB...,"[ormas, dan, dukungan, abal, masih, akan, ada,...","[ormas, dukungan, abal, lbp, moeldoko, mengera..."
9,Negatif,Semakin dahsyat untuk mencapai nafsu kekuasaan,"[semakin, dahsyat, untuk, mencapai, nafsu, kek...","[dahsyat, mencapai, nafsu, kekuasaan]"


In [53]:
#Case Folding
df['tweet'] = df['tweet'].str.lower()
df.head(10)

,Label,tweet,Tokenization,Stop_Removal
0,Netral,apa dia ngerti,"[apa, dia, ngerti]",[ngerti]
1,Netral,,[],[]
2,Negatif,suara sumbang ancam konstitusi marwah desa dii...,"[suara, sumbang, ancam, konstitusi, marwah, de...","[suara, sumbang, ancam, konstitusi, marwah, de..."
3,Negatif,banyak tong bahkan lbh baik siapa buka matamu ...,"[banyak, tong, bahkan, lbh, baik, siapa, buka,...","[tong, lbh, buka, matamu, gimmick, akalan, kau..."
4,Negatif,jokowi klaim usulan periode dari masyarakat ma...,"[jokowi, klaim, usulan, periode, dari, masyara...","[jokowi, klaim, usulan, periode, masyarakat, m..."
5,Negatif,istana ndableg kita tetap demo,"[istana, ndableg, kita, tetap, demo]","[istana, ndableg, demo]"
6,Positif,reformasi jilid menanti gantian periode,"[reformasi, jilid, menanti, gantian, periode]","[reformasi, jilid, gantian, periode]"
7,Positif,maju terus mhsiswa seluruh bersama rakyat panj...,"[maju, terus, mhsiswa, seluruh, bersama, rakya...","[maju, mhsiswa, rakyat, umur, perjuangan, peri..."
8,Negatif,ormas dan dukungan abal masih akan ada lagi lb...,"[ormas, dan, dukungan, abal, masih, akan, ada,...","[ormas, dukungan, abal, lbp, moeldoko, mengera..."
9,Negatif,semakin dahsyat untuk mencapai nafsu kekuasaan,"[semakin, dahsyat, untuk, mencapai, nafsu, kek...","[dahsyat, mencapai, nafsu, kekuasaan]"


In [54]:
# STEMMING
# buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['Stop_Removal']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# terapkan istilah bertangkai ke kerangka data
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['Tweet_Stemmed'] = df['Stop_Removal'].swifter.apply(get_stemmed_term)
print(df['Tweet_Stemmed'])

471
------------------------
ngerti : ngerti
 : 
suara : suara
sumbang : sumbang
ancam : ancam
konstitusi : konstitusi
marwah : marwah
desa : desa
diinjak : injak
injak : injak
kepala : kepala
desanya : desa
warga : warga
pemegang : pegang
kedaulatan : daulat
bersuara : suara
sikap : sikap
berpegang : pegang
teguh : teguh
kabar : kabar
bangsa : bangsa
tong : tong
lbh : lbh
buka : buka
matamu : mata
gimmick : gimmick
akalan : akal
kaum : kaum
togog : togog
pengin : pengin
nambah : nambah
periode : periode
utang : utang
menggunung : gunung
ekonomi : ekonomi
amburadul : amburadul
tdl : tdl
listrik : listrik
bbm : bbm
cukai : cukai
pajak : pajak
sembako : sembako
dll : dll
dinaikkan : naik
mikir : mikir
jokowi : jokowi
klaim : klaim
usulan : usul
masyarakat : masyarakat
nih : nih
istana : istana
ndableg : ndableg
demo : demo
reformasi : reformasi
jilid : jilid
gantian : ganti
maju : maju
mhsiswa : mhsiswa
rakyat : rakyat
umur : umur
perjuangan : juang
ugalan : ugalan
si : si
official : off

Pandas Apply:   0%|          | 0/115 [00:00<?, ?it/s]

0                                               [ngerti]
1                                                     []
2      [suara, sumbang, ancam, konstitusi, marwah, de...
3      [tong, lbh, buka, mata, gimmick, akal, kaum, t...
4      [jokowi, klaim, usul, periode, masyarakat, mas...
                             ...                        
110    [n, pantesan, getol, banget, tahan, periode, t...
111    [shaum, sabtu, ahad, tdk, shaum, lupa, agenda,...
112                                            [periode]
113    [ngurus, receh, ora, gableg, ngotot, kuasa, pe...
114                                            [periode]
Name: Tweet_Stemmed, Length: 115, dtype: object


In [55]:
df.to_csv("dataset_processing_clean.csv")

In [56]:
df1=pd.read_csv("dataset_processing_clean.csv")
df1

,Unnamed: 0,Label,tweet,Tokenization,Stop_Removal,Tweet_Stemmed
0,0,Netral,apa dia ngerti,"['apa', 'dia', 'ngerti']",['ngerti'],['ngerti']
1,1,Netral,NaN,[''],[''],['']
2,2,Negatif,suara sumbang ancam konstitusi marwah desa dii...,"['suara', 'sumbang', 'ancam', 'konstitusi', 'm...","['suara', 'sumbang', 'ancam', 'konstitusi', 'm...","['suara', 'sumbang', 'ancam', 'konstitusi', 'm..."
3,3,Negatif,banyak tong bahkan lbh baik siapa buka matamu ...,"['banyak', 'tong', 'bahkan', 'lbh', 'baik', 's...","['tong', 'lbh', 'buka', 'matamu', 'gimmick', '...","['tong', 'lbh', 'buka', 'mata', 'gimmick', 'ak..."
4,4,Negatif,jokowi klaim usulan periode dari masyarakat ma...,"['jokowi', 'klaim', 'usulan', 'periode', 'dari...","['jokowi', 'klaim', 'usulan', 'periode', 'masy...","['jokowi', 'klaim', 'usul', 'periode', 'masyar..."
...,...,...,...,...,...,...
110,110,Negatif,n pantesan getol banget mempertahankan periode...,"['n', 'pantesan', 'getol', 'banget', 'memperta...","['n', 'pantesan', 'getol', 'banget', 'memperta...","['n', 'pantesan', 'getol', 'banget', 'tahan', ..."
111,111,Netral,mau shaum sabtu atau ahad tdk jadi masalah pen...,"['mau', 'shaum', 'sabtu', 'atau', 'ahad', 'tdk...","['shaum', 'sabtu', 'ahad', 'tdk', 'shaum', 'lu...","['shaum', 'sabtu', 'ahad', 'tdk', 'shaum', 'lu..."
112,112,Netral,periode,['periode'],['periode'],['periode']
113,113,Negatif,ngurus hal receh ora gableg ngotot mau berkuas...,"['ngurus', 'hal', 'receh', 'ora', 'gableg', 'n...","['ngurus', 'receh', 'ora', 'gableg', 'ngotot',...","['ngurus', 'receh', 'ora', 'gableg', 'ngotot',..."
